In [26]:
from matplotlib import image
import os
import math
import matplotlib.pyplot as plt
import random
import numpy as np
import numpy.linalg as lin
import scipy.special as ss
from dataclasses import dataclass
import copy

# Convert binary text into a list
def convert_to_list(raw_img):
  converted_list = []
  temp_row = []
  for i in raw_img:
    if(i != '\n'):
      if(i == '0'):
        temp_row.append(0)
      else:
        temp_row.append(1)
    else:
      converted_list.append(temp_row)
      temp_row = []

  return converted_list

###############################################
# Displaying listed data as images
###############################################
def display_img(img_data):
  plt.imshow(img_data, cmap='Greys',  interpolation='nearest')
  plt.axis('off')
  return

###############################################
# Adding noise to the system
###############################################
def add_noise(data,noise_prob):

  # Initialize noisy data
  noisy_data = []

  # Sanity checker
  if(noise_prob < 0 or noise_prob > 1):
    print("Error! Noise probability isn't correct")
    return

  # Get total length per row
  col_length = len(data[0])

  # Generate fixed length
  shuffle_list = [x for x in range(col_length)]
  cutoff_idx = round(col_length * noise_prob)

  # Iterate per row
  for row in data:

    # Do random indexing
    random.shuffle(shuffle_list)
    temp_row = []

    # Start flipping bits
    for i in range(col_length):
      if(shuffle_list[i] < cutoff_idx):
        if(row[i] == 0):
          temp_row.append(1)
        else:
          temp_row.append(0)
      else:
        temp_row.append(row[i])

    noisy_data.append(temp_row)

  return noisy_data

###############################################
# This just displays a clean set of letters
###############################################
def show_set(clean_letters):

  fig, axs = plt.subplots(6, 5, figsize=(20, 20))

  counter = 0
  for i in range(5):
    for j in range(5):
      axs[i,j].imshow(1-np.reshape(clean_letters[i*5+j],(7,5)), cmap='Greys',  interpolation='nearest')

  axs[5,0].axis('off')
  axs[5,1].axis('off')

  axs[5,2].imshow(1-np.reshape(clean_letters[25],(7,5)), cmap='Greys',  interpolation='nearest')

  axs[5,3].axis('off')
  axs[5,4].axis('off')

  plt.show()

###############################################
# Displays a single letter
###############################################
def show_letter(letter):
  plt.imshow(1-np.reshape(letter,(7,5)), cmap='Greys',  interpolation='nearest')

###############################################
# Magnitude counter
###############################################
def get_mag(A):
  return np.sum(A)

###############################################
# Importing data
###############################################
# This data set contains all the letter from A to Z
# Each row is a vectorized version of the letter
# Each letter image has 7x5 pixel dimensions
# The data set is arranged such that A is the first row and Z is the last
# We made them into arrays too for simplicity

# clean_letters = convert_to_list(list(requests.get('https://raw.githubusercontent.com/rgantonio/CoE161---FileDump/main/letters.txt').text))
with open("char_data.txt") as data_file:
  clean_letters = convert_to_list(data_file.read())
clean_letters = np.array(clean_letters)


In [27]:
###############################################
# Set parameters
###############################################
D = 512
M = 10    # Number of ones
N = 3     # Sliding window size
m = 5     # Signature size
print("Density: " + str(M/D))

Density: 0.01953125


In [28]:
def u_gen_rand_hv(D):

    # Sanity checker
    if (D % 2):
        print("Error - D can't be an odd number")
        return 0

    hv = np.zeros(D, dtype = int)
    indices = np.random.permutation(D)

    hv[indices >= M] = 0
    hv[indices < M] = 1

    return hv

In [29]:
def overlap(A,B,D):
    and_out = np.logical_and(A,B)
    return np.sum(and_out)/D

In [30]:
def perm(A,N):
    return np.roll(A,N)

In [31]:
def superimpose(block):
    # Apply OR to all HV's in block
    return np.array([1 if x >= 1 else 0 for x in sum(block)])

In [32]:
def countones(Z):
    count = 0
    for i in Z:
        count += i
    return count

In [33]:
def CDT(Z,K=1):
    thinned_list = []
    for k in range(1,K+1):
        thinned_list.append(np.logical_and(Z, perm(Z,k)))
    res = superimpose(thinned_list)
    # print(countones(res)/D)
    return res

Import and export

In [34]:
def import_im_char(D):
  letter_im = dict()
  f = open("im_man.txt","r")
  for n in range(35):
    line = f.readline()
    line = list(line[-(D+2):-2])
    line = [int(x) for x in line]
    letter_im[n] = np.array(line)
  
  f.close()
  return letter_im

In [35]:
def import_am_char(D, keys):
  letter_am = dict()
  f = open("char_am.txt","r")
  for n in range(len(keys)):
    line = f.readline()
    line = list(line[0:1024])
    line = [int(x) for x in line]
    letter_am[keys[n]] = np.array(line)
  
  f.close()
  return letter_am

In [36]:
def import_am_lang(D, lang_labels):
  lang_am = dict()
  f = open("lang_am.txt","r")
  for n in range(len(lang_labels)):
    line = f.readline()
    line = list(line[0:D])
    line = [int(x) for x in line]
    lang_am[lang_labels[n]] = np.array(line)
  
  f.close()
  return lang_am

# Encoding

In [37]:
# Dataclass containing the encoder programming
@dataclass
class Encodersettings:
    sliding_window_mode: bool = 0
    signature_encoding_mode: bool = 0
    xor_binding_mode: bool = 0
    shift_binding_mode: bool = 0
    shift_amount: int = 0

    or_mode: bool = 0
    acc1_mode: bool = 0
    cdt_mode: bool = 0
    acc2_mode: bool = 0
    cdt_k_fact: int = 1
    thr1_val: int = 0
    thr2_val: int = 0
    window1_size: int = 3


In [38]:
def binarize_hv_thr(v, threshold):
    
    for i in range(len(v)):
        if v[i] >= threshold:
            v[i] = 1
        else:
            v[i] = 0
    return v

In [39]:
def buffer_to_binary(buffer):
  letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',' ','\n']
  lang_data = ""
  for i in range(len(buffer)):
    if (buffer[i] not in letters):
      print("Error: unknown letter encountered")
      return None
    idx = letters.index(buffer[i])
    bin_idx = bin(idx)[2:].zfill(6) # removes "0b" and adds zeros in front
    lang_data = lang_data + bin_idx
    if (i != len(buffer)-1):
      lang_data += "\n"
  return lang_data

In [40]:
def encoder(input_buffer, shift_amount_buffer, item_mem, D, enc_set):
    window1_cnt = enc_set.window1_size
    acc1_regs = np.zeros(D, dtype = int)
    acc2_regs = np.zeros(D, dtype = int)
    reg1_out = np.zeros(D, dtype = int)
    win_counter = 0
    in_value_queue = np.zeros(enc_set.window1_size, dtype = int)
    i = -1
    input_done = 0
    
    while not input_done:
        
        # BINDING
        # lang_recog signature encoding
        if (enc_set.sliding_window_mode):
            enc_set.shift_binding_mode = 1
            if (win_counter < (enc_set.window1_size-1) ) and i != -1:
                win_counter += 1
            else:
                win_counter = 0
                i += 1
                input_value = input_buffer[i]
                in_value_queue = np.insert(in_value_queue, 0, input_value)
                in_value_queue = np.delete(in_value_queue,-1)
            
            in_value_mapper = in_value_queue[win_counter]
            xor_tot = 0
            if (enc_set.signature_encoding_mode):
                for j in range(0,enc_set.window1_size):
                    if (j != win_counter):
                        xor_tot = np.bitwise_xor(xor_tot, in_value_queue[j])
            shift_amount = win_counter + xor_tot
            input_done = (i == len(input_buffer)-1) and (win_counter == enc_set.window1_size-1)
            
            im_en = (win_counter <= i)
            
        else:
            i += 1
            input_done = (i == len(input_buffer)-1)
            input_value = input_buffer[i]

            in_value_mapper = input_value
            shift_amount = shift_amount_buffer[i]
            im_en = 1

        # Fetch HV from IM and CIM

        if (im_en):
            im_out = item_mem[in_value_mapper]
        else:
            im_out = np.zeros(D, dtype=int)
        # cim_out = continuous_item_mem[in_value_mapper]
        if (enc_set.xor_binding_mode):
            # xor_out = np.logical_xor(im_out, cim_out)
            xor_out = None
        else:
            xor_out = im_out

        # Permutation
        if (not enc_set.shift_binding_mode):
            binding_out = xor_out
        else:
            binding_out = perm(xor_out, shift_amount)

        # BUNDLING
        thr1_out = np.zeros(D, dtype = int)
        if (enc_set.or_mode):
            if (input_done):
                acc1_regs = np.logical_or(acc1_regs, binding_out)
                window1_cnt -= 1
                reg1_out = copy.deepcopy(acc1_regs)
                stage1_done = 1
            elif (window1_cnt > 0):
                acc1_regs = np.logical_or(acc1_regs, binding_out)
                window1_cnt -= 1
                stage1_done = 0
            else:
                reg1_out = copy.deepcopy(acc1_regs)
                window1_cnt = enc_set.window1_size - 1
                acc1_regs = binding_out
                stage1_done = 1

        elif (enc_set.acc1_mode):
            if (input_done):
                acc1_regs += binding_out
                window1_cnt -= 1
                thr1_out = binarize_hv_thr(copy.deepcopy(acc1_regs), enc_set.thr1_val)
                stage1_done = 1
            elif (window1_cnt > 0):
                acc1_regs += binding_out
                window1_cnt -= 1
                stage1_done = 0
            else:
                thr1_out = binarize_hv_thr(copy.deepcopy(acc1_regs), enc_set.thr1_val)
                window1_cnt = enc_set.window1_size - 1
                acc1_regs = binding_out
                stage1_done = 1
            reg1_out = thr1_out
        else:
            reg1_out = binding_out
            stage1_done = 1

        #Stage 2 bundling
        thr2_out = np.zeros(D, dtype = int)
        if (stage1_done):
            if (enc_set.cdt_mode):
                cdt_out = CDT(reg1_out,enc_set.cdt_k_fact)
            else:
                cdt_out = reg1_out

            if (enc_set.acc2_mode):
                acc2_regs += cdt_out
                if (input_done):
                    thr2_calc = enc_set.thr2_val * (i+1) * (2**(-10))
                    thr2_out = binarize_hv_thr(copy.deepcopy(acc2_regs), thr2_calc)
                reg2_out = thr2_out
            else:
                reg2_out = cdt_out
    
    return reg2_out
            
    



In [41]:
# Character recognition
encoded_lst = []
for i in range(len(clean_letters)):
    input_buffer = list(range(35))
    letter = clean_letters[i]
    shift_amount_buffer = []
    for i in letter:
        shift_amount_buffer.append(int(i))
    # print(shift_amount_buffer)

    item_mem = import_im_char(D)
    enc_set_char = Encodersettings(
        shift_binding_mode=1,
        or_mode=1,
        cdt_mode=1,
        cdt_k_fact=1,
        window1_size=35)

    encoded_hv = encoder(input_buffer, shift_amount_buffer, item_mem, D, enc_set_char)
    encoded_hv = [x for x in encoded_hv]
    # print(encoded_hv)
    encoded_lst.append(encoded_hv)

# Check results with encoded HVs from char_testvector_gen
# f = open("char_after_cdt.txt","r")
# for j in range(len(clean_letters)):
#     line = f.readline()
#     line = list(line[0:D])
#     line = [int(x) for x in line]

#     for i in range(len(encoded_hv)):
#         if (encoded_lst[j][i] != line[i]):
#             print("Error: letter " + str(j) + "pos: " + str(i))

# f.close()


FileNotFoundError: [Errno 2] No such file or directory: 'im_man.txt'

In [ ]:
# Language recognition
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',' ','\n']
text = "hello this is a test"
shift_amount_buffer = np.zeros(len(text),dtype=int)
input_buffer = [letters.index(x) for x in text]
# print(input_buffer)

item_mem = import_im_char(D)
enc_set_lang = Encodersettings(
    sliding_window_mode=1,
    signature_encoding_mode=1,
    acc1_mode=1,
    acc2_mode=1,
    window1_size=3,
    thr1_val = 2,
    thr2_val = 20)

encoded_hv = encoder(input_buffer, shift_amount_buffer, item_mem, D, enc_set_lang)
encoded_hv = [x for x in encoded_hv]
# print(encoded_hv)

# Check results with encoded HVs from lang_testvector_gen
f = open("lang_encoded.txt","r")
line = f.readline()
line = list(line[0:D])
line = [int(x) for x in line]

for i in range(len(encoded_hv)):
    if (encoded_hv[i] != line[i]):
        print("Error: letter pos: " + str(i))

f.close()

In [ ]:
# Mixed testbench
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',' ','\n']
text = "executing mixed test"
mixed_data = buffer_to_binary(text)
if mixed_data is None:
    print("Error")
f = open("mixed_data.txt", "w")
f.write(mixed_data)
f.close()
shift_amount_buffer = np.zeros(len(text),dtype=int)
input_buffer = [letters.index(x) for x in text]
# print(input_buffer)

f = open("mixed_data_encoded.txt", "w").close()
item_mem = import_im_char(D)
def mixed_test(item_mem, enc_set):
  encoded_hv = encoder(input_buffer, shift_amount_buffer, item_mem, D, enc_set)
  encoded_hv = [x for x in encoded_hv]
  # print(encoded_hv)
  print("Density: " + str(countones(encoded_hv)/D))
  hv = "".join(str(x) for x in encoded_hv)
  f = open("mixed_data_encoded.txt", "a")
  f.write(str(hv) + "\n")
  f.close()
  return

# Mixed tests
# Lang with other input and thr
enc_set_mixed = Encodersettings(
  sliding_window_mode=1,
  signature_encoding_mode=1,
  acc1_mode=1,
  acc2_mode=1,
  window1_size=3,
  or_mode=0,
  cdt_mode=0,
  cdt_k_fact=1,
  thr1_val=2,
  thr2_val=100)
mixed_test(item_mem, enc_set_mixed)

# Max sliding window size
enc_set_mixed = Encodersettings(
  sliding_window_mode=1,
  signature_encoding_mode=1,
  acc1_mode=1,
  acc2_mode=1,
  window1_size=12,
  or_mode=0,
  cdt_mode=0,
  cdt_k_fact=1,
  thr1_val=2,
  thr2_val=100)
mixed_test(item_mem, enc_set_mixed)

# Sliding window without signature encoding
enc_set_mixed = Encodersettings(
  sliding_window_mode=1,
  signature_encoding_mode=0,
  acc1_mode=1,
  acc2_mode=1,
  window1_size=4,
  or_mode=0,
  cdt_mode=0,
  cdt_k_fact=1,
  thr1_val=2,
  thr2_val=20)
mixed_test(item_mem, enc_set_mixed)

# Other window and or_mode instead of acc1
enc_set_mixed = Encodersettings(
  sliding_window_mode=1,
  signature_encoding_mode=1,
  acc1_mode=0,
  acc2_mode=1,
  window1_size=2,
  or_mode=1,
  cdt_mode=0,
  cdt_k_fact=1,
  thr1_val=2,
  thr2_val=100)
mixed_test(item_mem, enc_set_mixed)

# or_mode and cdt_mode followed by acc2
enc_set_mixed = Encodersettings(
  sliding_window_mode=1,
  signature_encoding_mode=1,
  acc1_mode=0,
  acc2_mode=1,
  window1_size=2,
  or_mode=1,
  cdt_mode=1,
  cdt_k_fact=1,
  thr1_val=2,
  thr2_val=50)
mixed_test(item_mem, enc_set_mixed)

# Only cdt, no bundling in stage1
enc_set_mixed = Encodersettings(
  sliding_window_mode=0,
  signature_encoding_mode=0,
  acc1_mode=0,
  acc2_mode=1,
  window1_size=1,
  or_mode=0,
  cdt_mode=1,
  cdt_k_fact=1,
  thr1_val=2,
  thr2_val=50)
mixed_test(item_mem, enc_set_mixed)

# Only stage 2 bundling, no bundling in stage1
enc_set_mixed = Encodersettings(
  sliding_window_mode=0,
  signature_encoding_mode=0,
  acc1_mode=0,
  acc2_mode=1,
  window1_size=1,
  or_mode=0,
  cdt_mode=0,
  cdt_k_fact=1,
  thr1_val=2,
  thr2_val=100)
mixed_test(item_mem, enc_set_mixed)

# Only s1 cdt, s2 bundling, with cdt_k changed
enc_set_mixed = Encodersettings(
  sliding_window_mode=0,
  signature_encoding_mode=0,
  acc1_mode=0,
  acc2_mode=1,
  window1_size=1,
  or_mode=0,
  cdt_mode=1,
  cdt_k_fact=3,
  thr1_val=2,
  thr2_val=50)
mixed_test(item_mem, enc_set_mixed)

# or_mode and cdt_mode followed by acc2, cdt_k changed
enc_set_mixed = Encodersettings(
  sliding_window_mode=1,
  signature_encoding_mode=1,
  acc1_mode=0,
  acc2_mode=1,
  window1_size=2,
  or_mode=1,
  cdt_mode=1,
  cdt_k_fact=2,
  thr1_val=2,
  thr2_val=50)
mixed_test(item_mem, enc_set_mixed)


Density: 0.009765625
Density: 0.41552734375
Density: 0.19140625
Density: 0.546875
Density: 0.13427734375
Density: 0.02294921875
Density: 0.248046875
Density: 0.076171875
Density: 0.24951171875
